# Dataset Preparation | Detecting the products

In [151]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os

from PIL import Image
from warnings import filterwarnings as w; w('ignore')


## Define Paths

In [ ]:
#paths to the saved model and label map
my_model = 'ssd_mobilenet_v2'
training_path = "Tensorflow/workspace/Training/"
exported_model_path = training_path + 'exported_models/' + my_model +'/'
label_path = training_path + 'annotations/label_map.pbtxt'

## Change the 'name' tag in all xml files to 'product'.

In [ ]:

cols = ['img_name', 'xmin','ymin','xmax','ymax', 'dntknow']
xmls_paths = 'annotations/newst/train/'
xml_name = os.listdir(xmls_paths)
xml_names = []
for file in xml_name:
    if file.endswith(".xml"):
        xml_names.append(file)
        
import xml.etree.ElementTree as et


for file in xml_names:
    tree = et.parse(xmls_paths + file)
    root = tree.getroot()
    for i in root.findall('object'):
        name = i.find('name')
        name.text = 'product'
    tree.write(xmls_paths+file)
 

## Label Map: (1 detection class)

In [ ]:
annot_path = 'Tensorflow/workspace/Training/annotations/'

labels = [{'name':'product', 'id':1}]

with open(annot_path +'label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item{\n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## Generate tf records 
* For the train and test set we created

In [ ]:
%cd Tensorflow/scripts
!wget https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py   
%cd ../..    

* edit the 'class_text_to_int' function in the above script to fit the product class

In [ ]:
#Train set
!python Tensorflow/scripts/generate_tfrecord.py -x {training_path+ 'images/train'} -l {label_path} -o {training_path+'annotations/train.record'}

#Test set
!python Tensorflow/scripts/generate_tfrecord.py -x {training_path+ 'images/test'} -l {label_path} -o {training_path+'annotations/test.record'}



* The train.record and test.record binary files are copied to drive for training

## Function to write image to disk

In [ ]:
def write(name, img):
    to = 'trial/images' + name 
    print("Writing to Disk: ", to)
    cv2.imwrite(to, img)

## Dependancies for custom detection

In [156]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
import time
import tensorflow as tf

## Loading the trained model

In [218]:
#Paths to the test set images
from imutils import paths
image_paths= list(paths.list_images('ShelfImages/test'))
image_paths.sort


PATH_TO_SAVED_MODEL = exported_model_path + "saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))


#load label_map
category_index = label_map_util.create_category_index_from_labelmap(label_path, use_display_name=True)

dicto = {}


Loading model...Done! Took 17.23454999923706 seconds


In [252]:
def load_image_into_np_array(path):
    return np.array(Image.open(path))

## Detection

In [290]:
def detectboxes(detect_fn, path, thresh):    
    image_np = load_image_into_np_array(path)
    img_name = path.split(os.path.sep)[-1]
    
    input_tensor = tf.convert_to_tensor(image_np)

    #THe function accepts a batch
    input_tensor = np.expand_dims(input_tensor, 0)

    #function created from saved model
    detections = detect_fn(input_tensor)
    
    #no of boxes that will be drawn based on threshold
    n = np.array(detections['detection_scores'])
    num = len(n[n>thresh])
    
    
    num_detections = int(detections.pop('num_detections')) #100 detections
    
    
    #Retain 100 detections only
    detections = {key:value[0, :num_detections].numpy() for key,value in detections.items()}

    #store num_detections back
    detections['num_detections'] = num_detections
    

    #detection classes should be ints
    detections['detection_classes'] = detections['detection_classes'].astype('int64')
    
    image_np_with_detections = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index, 
        use_normalized_coordinates = True,
        max_boxes_to_draw = 200,
        min_score_thresh= thresh,
        agnostic_mode=False,
        line_thickness = 10)
    
    
    dicto.update({img_name: num})
    
    write(img_name, image_np_with_detections)
    #plt.figure(figsize = (12,10))
    #plt.imshow(image_np_with_detections)
    #print('Done')
    return dicto
    


## Running through  all the images in the test set

In [ ]:
cnt = 0
for path in image_paths:
    dic = detectboxes(detect_fn, path , 0.5301)
    cnt += 1
    print("Image no: ", cnt)

#Write image2products JSON to disk
with open('trial2/image2products.json', 'w') as f:
    json.dump(dic, f, indent = 4)
    

#Write the metrics got from evaluation to a JSON file:
#metrics = {'mAP': 0.669, 'Precision':0.670, 'Recall': 0.726}
#with open('Output Files/metrics.json', 'w') as f:
#    json.dump(met, f, indent = 4)

# THE END

In [9]:
from collections import Counter
s ='{{([]}}}'

a = dict(Counter(s))


for k,v in a.items():
    if k =='{':
        if a['}'] < v:
            print('} is missing for a respective {')
            
        elif a['}'] > v:
            print('{ is missing for a respective }')
    elif k == '[':
        if a[']'] < v:
            print('] is missing for a respective [')
        elif a[']'] > v:
            print('[ is missing for a respective ]')
            
    elif k == '(':
        if ')' in a:
            if a[')'] < v:
                print(') is missing for a respective (')
            elif a[')'] > v:
                print('( is missing for a respective )')
        else:
            print(') is missing for a respective (')

{ is missing for a respective }
) is missing for a respective (


In [2]:
{ { ( [ ] } } }

2

In [11]:
m = [1,2,3,4,5]
m.pop()
m

[1, 2, 3, 4]